# Excel Testing Notebook



In [1]:
#pull in some useful modules
using DataFrames #DataFrames is based on similar concepts in R and Python and gives us indexed tables with headers
using Dates

## Read_Generic_Table style lookup

Implimented as a macro so a bunch of work is done a compile time. Much like Read_Generic_Table this means a single code entry can be used to lookup any table with any number of indexes. The macro reformats this at compile time into the necessary DataFrames calls. It's not especially easy to follow, and there's no real way to ever see the code which is generated.. but arguably the same is true of Read_Generic_Table in Prophet.

In [2]:
macro TableLookup(t,returnColumn,indexes...)
    for(ind,val) in enumerate(indexes) #loop through the list of supplied index conditions...
        if(ind==1)
            exp= :($t[$ind].==$val)
        else
            exp = :($exp & ($t[$ind].==$val)) #... and create a Julia expression of the form
                                              # tableName[indexNum]==indexVal]
        end
        #@show(exp)
    end

    exp = :($t[$exp,Symbol(DataFrames.makeidentifier($returnColumn))][1]) ##final Julia expression to return
    #exp = :($t[$exp,:])
    #@show(exp)
end
    

@TableLookup (macro with 1 method)

## Load some basis tables

Broadly its bad practice to define anything as a global, but we will here for simplicity

In [3]:
ilq_Table = readtable("basTest_ilq.csv")
rfr_Table = readtable("basTest_rfr.csv")

,DUR,x201412
1,0,0.384695331
2,1,0.384695331
3,2,0.408181475
4,3,0.431706662
5,4,0.455290414
6,5,0.478952254
7,6,0.502711704
8,7,0.526586786
9,8,0.550580543
10,9,0.574676539


## inline function defintion

in Julia it's possible to define simple functions in the form `f(x) = 2x+1` which we use here to define two versions of the conversion from a yeild curve data to discrete spot rates. (really one would do but this demonstrates multiple dispatch a bit)

In [4]:
ann_spot_rate(foi) = e^(foi/100)-1
ann_spot_rate(foi,illiqPC) = (e^(foi/100)-1)*(illiqPC/100)

ann_spot_rate (generic function with 2 methods)

## "Dumb" Conversion of v_Calc_Interest_Rates

This version of the calculation approaches things like the excel model = everything is an array, and everything happens within one big loop, much like Prophet. It also does a bunch of work it doesn't need to where `spot_rate_freq` is set to something other than 12, because everything between `vsr` and `mfr` is essentially thrown away in the non-calc months, but I don't believe there's any way when converting the Excel formula to necessarily anticipate this.

In [5]:
function v_interest_rates1(maxTerm::Integer,yeildCurve::Integer,spot_rate_freq::Integer,illiqPC::Float64)
    
    #Array initialisation. Reasonable to expect we can antipate this from the Excel structure
    rfr = zeros(maxTerm)
    ilq = zeros(maxTerm)
    vsr = zeros(maxTerm)
    calc_forward_rate = zeros(maxTerm)
    t1 = zeros(maxTerm)
    t2 = zeros(maxTerm)
    s1 = zeros(maxTerm)
    s2 = zeros(maxTerm)
    fra = zeros(maxTerm)
    mfr = zeros(maxTerm)    
   
    #Column E
    for t in 1:maxTerm #for each row...

        #lookup risk free rate and illiquidity values and perfom the spot rate calc. The form here would only be possible
        #if we'd also implimented as ann_spot_rate function in Excel
        
        #Columns F and G
        rfr[t] = ann_spot_rate(@TableLookup(rfr_Table,string(yeildCurve),t))        
        
        #Columns H and I
        ilq[t] = ann_spot_rate(@TableLookup(ilq_Table,string(yeildCurve),t),illiqPC)
        
        #Column J
        vsr[t]=rfr[t]+ilq[t]
        
        #Column K
        if mod((t-1)*spot_rate_freq,12)==0
            calc_forward_rate[t]=1
        else
            calc_forward_rate[t]=0
        end
        
        #Column L
        if calc_forward_rate[t]==1
            t1[t]=(t-1)/12
        else
            t1[t]=0
        end
        
        #Column M
        if calc_forward_rate[t]==1
            t2[t]=t/12
        else
            t2[t]=0
        end
        
        #Column N
        if t>1
          s1[t]=vsr[t-1]
        else
          s1[t]=vsr[t]
        end
        
        #Column O
        s2[t] = vsr[t]
        
        #Column P
        if calc_forward_rate[t]==1
            fra[t] = ((1+s2[t])^t2[t]/(1+s1[t])^t1[t])^(1/(t2[t]-t1[t]))-1
        else
            fra[t]= fra[t-1]
        end
        
        #Column Q
        mfr[t] = (1+fra[t])^(1/12)-1
    end
    
    #Assume we've specified somehow that this is the value from the see we want available elsewhere
    mfr
end

v_interest_rates1 (generic function with 1 method)

## "Hand-coded" Conversion of  v_Calc_Interest_Rates

This version recognises that the entire yeild curve is required at some point, so retrieves it in one go (this is probably where the majority of the performance benefit is). It also does a better job of avoiding uncessary calculation and memory use by not retaining transient data and skipping the entire process on non-calc months.

In [6]:
function v_interest_rates2(maxTerm::Integer,yeildCurve::Integer,spot_rate_freq::Integer,illiqPC::Float64)

    forward_rate = zeros(maxTerm)
    monthly_forward_rate = zeros(maxTerm)
    vsr = zeros(maxTerm)

    #get the entire column from the table. There's probably a more reable way to do this
    rfr_fofi = rfr_Table[Symbol(DataFrames.makeidentifier(string(yeildCurve)))] 
    ilq_fofi = ilq_Table[Symbol(DataFrames.makeidentifier(string(yeildCurve)))]

    for t in 1:maxTerm

        #potentially could make ann_spot_rate work on vectors and move this outside the loop
        vsr[t] = ann_spot_rate(rfr_fofi[t]) + ann_spot_rate(ilq_fofi[t],illiqPC)

        if t==1
            forward_rate[t] = vsr[t]
        elseif mod((t-1)*spot_rate_freq,12)==0
            t1 = (t-1)/12
            t2 = t/12
            s1 = vsr[t-1]
            s2 = vsr[t]

            forward_rate[t] = ((1+s2)^t2/(1+s1)^t1)^(1/(t2-t1))-1

        else
            forward_rate[t] = forward_rate[t-1]
        end
        
        monthly_forward_rate[t] = (1+forward_rate[t])^(1/12)-1
    end

    monthly_forward_rate
end

v_interest_rates2 (generic function with 1 method)

### Performance Testing

v2 function is significantly quicker, and scales better - at 50 iterations is around 3x faster, on this 720 iteration loop it's over 10x faster. 

In [13]:
maxDur = maximum(rfr_Table[:DUR])

@time v_interest_rates1(maxDur,201412,12,0.0)
@time v_interest_rates2(maxDur,201412,12,0.0)

  0.008065 seconds (42.91 k allocations: 2.974 MB)
  0.000653 seconds (8.36 k allocations: 148.438 KB)


720-element Array{Float64,1}:
 0.000320631
 0.000320631
 0.000379367
 0.000418656
 0.000458126
 0.000497842
 0.000537868
 0.000578261
 0.000618967
 0.000659835
 0.00070071 
 0.000741444
 0.000781887
 ⋮          
 0.00225304 
 0.00225776 
 0.00226248 
 0.00226722 
 0.00227197 
 0.00227673 
 0.0022815  
 0.00228629 
 0.00229108 
 0.00229588 
 0.00230069 
 0.00230551 

In [8]:
MaximumAge = 120
ValuationDate = Date(2014,12,31)

2014-12-31

### Custom Types Objects

Might be worthwhile to define composite types for the data items which as associated: e.g define a "Life" type/object which describes the details of a life assured. Some interim step would break up our single "MPF" data structure into these more object-like groups.

This would later enable us to move away from our monolith MPF structure to a more relational/structured data approach.

In [9]:
type LifeAssured   
    AGE_AT_ENTRY::Float64
    SMOKER_STAT::Int64
    SEX::Int64
end

#Create a "LifeAssured" called L1
L1 = LifeAssured(25.2,0,1)

LifeAssured(25.2,0,1)

In [10]:
# Can access members of "L1" using standard "dot notation" similar to VB/C#/etc
L1.AGE_AT_ENTRY, L1.SEX == 1

(25.2,true)

In [11]:
type PolDurations
    DURATIONIF_M::Int64
    POL_TERM_M::Int64
end